In [2]:
import pandas as pd
import numpy as np
import statistics


def median_of_ratios(data):
    d = pd.DataFrame(data.loc[data.min(axis=1) > 0])
    pseudo_column = []
    for gene, row in d.iterrows():
        pseudo_column.append(statistics.geometric_mean(row))
    d['pseudo'] = pseudo_column
    return d.apply(lambda x: x / d['pseudo']).drop('pseudo', axis=1).\
        apply(lambda x: x.median()).values


df = pd.read_csv('TCGA-COAD_cancer_normal.tsv', sep='\t', index_col=0)
gl = pd.read_csv("gene_lengths.tsv", sep="\t", index_col=0).sort_index()

size_factors = median_of_ratios(df)

RPM = df.div(df.sum(axis=0), axis=1) * 1e+6
DESeq2_RPM = RPM.div(size_factors, axis=1)
DESeq2_RPKM = DESeq2_RPM.div(gl["Length"], axis=0) * 1000
DESeq2_RPKM = np.log2(DESeq2_RPKM + 1)

# характеризуем величину экспрессии через медиану
median = DESeq2_RPKM.median(axis=1)
highly_expressed = median.sort_values(ascending=False).iloc[:10]

print(f'The most highly expressed genes: {", ".join(list(highly_expressed.index))}')

# 8 из топ-10 генов принадлежат к митохондриальным генам, которые кодируют
# субъединицы цитохром оксидазы (MT-CO2, MT-CO1, MT-CO3), NADH-дегидрогеназы
# (MT-ND4, MT-ND3), АТФ-синтазы (MT-ATP6), 16S рРНК (MT-RNR2), убихинол-цитохром
# с-оксидоредуктаза (MT-CYB)

The most highly expressed genes: MT-CO2, MT-CO1, MT-CO3, MT-ND4, MT-ATP6, MT-RNR2, MT-ND3, FTL, IGHA1, MT-CYB
